In [51]:
import scipy
from sklearn.neighbors import KDTree
from scipy.spatial import distance 
import numpy as np
from numpy.random import default_rng
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import random
import copy
import functools

In [54]:
from functools import cache

ImportError: cannot import name 'cache' from 'functools' (/home/ois/anaconda3/lib/python3.8/functools.py)

In [2]:
from nltk.corpus import words
from nltk.corpus import wordnet 

In [3]:
words = list(wordnet.words())

def is_word(word):
    # NOTE: Removing hyphenated words too.
    return (
        not any(c in word for c in '-_. \'"0123456789,<>!@#$%^&*({[]})')
    )
print(len(words))

words = [w for w in words if is_word(w)]
print(len(words))
# Also convert everything to lower case... Maybe I don't want place names though..??
words = { w.lower() for w in words }
print(len(words))

147306
77510
77510


In [4]:
words_by_length = defaultdict(list)

for w in words:
    words_by_length[len(w)].append(w)

In [56]:
# TODO: Memoize
@functools.lru_cache(maxsize=None)
def words_from_constraint(length, position, letter):
    return { w for w in words_by_length[length] if w[position] == letter }

def words_from_constraints(length, constraints):
    if not constraints:
        return words_by_length[length]
    
    sets = [words_from_constraint(length, position, letter)
            for position, letter in constraints]
    
    return set.intersection(*sets)


words_from_constraints(5, [(0, 'a'), (3, 'b')])

{'adobe', 'adobo', 'akaba', 'alibi', 'ameba', 'aqaba', 'aruba'}

In [59]:
# A word is just a list of positions that're covered by that word in the grid.
# Could store 'position', 'direction' but...

# Generate all words in a grid

size_x = 5  # Width
size_y = 5

# | 0,0 | 1,0 | ..
# | 0,1 | 1,1 | ..
#   ..    ..

words = []

for i in range(size_x):
    words.append(
        [(i, j) for j in range(size_y)]
    )
    
for j in range(size_y):
    words.append(
        [(i, j) for i in range(size_x)]
    )
    
random.shuffle(words)

words

[[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4)],
 [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(4, 0), (4, 1), (4, 2), (4, 3), (4, 4)],
 [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)],
 [(0, 3), (1, 3), (2, 3), (3, 3), (4, 3)],
 [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2)],
 [(3, 0), (3, 1), (3, 2), (3, 3), (3, 4)],
 [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4)],
 [(2, 0), (2, 1), (2, 2), (2, 3), (2, 4)]]

In [60]:
def nice_print(grid):
    print('+-' * len(grid[0]) + '+')
    for row in grid:
        print('|' + '|'.join(row) + '|')
        print('+-' * len(row) + '+')
        
    print()

In [61]:
grid = [[' ' for i in range(size_x)] for j in range(size_y)]
# NOTE: Accessed with grid[y][x]

node_tracker = defaultdict(int)

def dfs(depth):
    node_tracker[depth] += 1
    
    if depth >= len(words):
        yield copy.deepcopy(grid)
        return
        
    # Build constraints for this word.
    new_letters = []
    constraints = []
    for ix, (i,j) in enumerate(words[depth]):
        c = grid[j][i]
        if c == ' ':
            new_letters.append((ix, i, j))
        else:
            constraints.append((ix, c))

#     print(new_letters, constraints, ix)

    # Find words.
    possible_words = words_from_constraints(ix+1, constraints)
    #print('found:', len(possible_words), 'possible words')
    #print(ix+1, constraints)
    for w in possible_words:
        # Insert word into grid...
        for ix, i, j in new_letters:
            grid[j][i] = w[ix]
            
        # Recurse
        for ans in dfs(depth + 1):
            yield ans
        
        # Remove word from grid
        for ix, i, j in new_letters:
            grid[j][i] = ' '
      
x = dfs(0)
for i in range(100):
    nice_print(next(x))

+-+-+-+-+-+
|h|a|l|m|a|
+-+-+-+-+-+
|i|l|i|o|n|
+-+-+-+-+-+
|j|a|n|u|s|
+-+-+-+-+-+
|a|m|u|s|e|
+-+-+-+-+-+
|b|o|x|e|r|
+-+-+-+-+-+

+-+-+-+-+-+
|j|a|c|o|b|
+-+-+-+-+-+
|a|d|o|b|o|
+-+-+-+-+-+
|c|o|d|e|x|
+-+-+-+-+-+
|o|b|e|s|e|
+-+-+-+-+-+
|b|o|x|e|r|
+-+-+-+-+-+

+-+-+-+-+-+
|j|a|c|o|b|
+-+-+-+-+-+
|a|d|o|b|o|
+-+-+-+-+-+
|c|o|d|e|r|
+-+-+-+-+-+
|o|b|e|s|e|
+-+-+-+-+-+
|b|o|x|e|r|
+-+-+-+-+-+

+-+-+-+-+-+
|h|a|l|a|l|
+-+-+-+-+-+
|a|d|a|n|a|
+-+-+-+-+-+
|l|o|t|i|c|
+-+-+-+-+-+
|a|b|e|l|e|
+-+-+-+-+-+
|b|o|x|e|r|
+-+-+-+-+-+

+-+-+-+-+-+
|h|a|l|a|b|
+-+-+-+-+-+
|a|l|a|m|o|
+-+-+-+-+-+
|l|a|r|i|x|
+-+-+-+-+-+
|a|m|i|n|e|
+-+-+-+-+-+
|b|o|x|e|r|
+-+-+-+-+-+

+-+-+-+-+-+
|h|a|l|a|b|
+-+-+-+-+-+
|a|l|a|m|o|
+-+-+-+-+-+
|l|a|s|i|x|
+-+-+-+-+-+
|a|m|i|n|e|
+-+-+-+-+-+
|b|o|x|e|r|
+-+-+-+-+-+

+-+-+-+-+-+
|h|a|l|a|b|
+-+-+-+-+-+
|a|l|a|m|o|
+-+-+-+-+-+
|l|a|r|i|x|
+-+-+-+-+-+
|a|m|i|d|e|
+-+-+-+-+-+
|b|o|x|e|r|
+-+-+-+-+-+

+-+-+-+-+-+
|h|a|l|a|b|
+-+-+-+-+-+
|a|l|a|m|o|
+-+-+-+-+-+
|l|a|s|i|

KeyboardInterrupt: 

In [58]:
node_tracker  

defaultdict(int,
            {0: 1,
             1: 1,
             2: 6,
             3: 1473,
             4: 314257,
             5: 45991660,
             6: 736266,
             7: 6743,
             8: 100})

In [ ]:
# +-+-+-+-+-+
# |h|a|r|s|h|
# +-+-+-+-+-+
# |a|m|a|t|i|
# +-+-+-+-+-+
# |l|i|d|a|r|
# +-+-+-+-+-+
# |a|g|i|l|e|
# +-+-+-+-+-+
# |b|o|x|e|r|
# +-+-+-+-+-+